In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# Define the seed
SEED = 26  # You can use any random seed value here

# Load the data
features = pd.read_excel('高温合金数据集合2.xlsx')
label1 = np.array(features['Creep life'])
features = features.drop(['UTS', 'TYS', 'EL'], axis=1)
label1 = np.log(label1)  # Take the logarithm
features = pd.get_dummies(features)
feature_list = list(features.columns)
features = np.array(features)
features = MinMaxScaler().fit_transform(features)

# Linear Regression
# Define the Linear Regression model
model_lr = LinearRegression()

# Define the number of folds
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=SEED)

# Initialize variables to store performance metrics
rmse_tr_lr_list = []
rmse_te_lr_list = []
r2_tr_lr_list = []
r2_te_lr_list = []

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(features):
    train_features, test_features = features[train_index], features[test_index]
    train_labels, test_labels = label1[train_index], label1[test_index]

    # Fit the Linear Regression model
    model_lr.fit(train_features, train_labels)

    # Predictions
    train_predicted_creeplife = model_lr.predict(train_features)
    test_predicted_creeplife = model_lr.predict(test_features)

    # Calculate performance metrics
    rmse_tr_lr = mean_squared_error(train_labels, train_predicted_creeplife, squared=False)
    rmse_te_lr = mean_squared_error(test_labels, test_predicted_creeplife, squared=False)
    r2_tr_lr = r2_score(train_labels, train_predicted_creeplife)
    r2_te_lr = r2_score(test_labels, test_predicted_creeplife)

    # Append the metrics to the lists
    rmse_tr_lr_list.append(rmse_tr_lr)
    rmse_te_lr_list.append(rmse_te_lr)
    r2_tr_lr_list.append(r2_tr_lr)
    r2_te_lr_list.append(r2_te_lr)

# Calculate the mean and standard deviation of the metrics
mean_rmse_tr_lr = np.mean(rmse_tr_lr_list)
mean_rmse_te_lr = np.mean(rmse_te_lr_list)
mean_r2_tr_lr = np.mean(r2_tr_lr_list)
mean_r2_te_lr = np.mean(r2_te_lr_list)

# Print the results for Linear Regression
print("Mean RMSE (Train): {:.4f}".format(mean_rmse_tr_lr))
print("Mean RMSE (Test): {:.4f}".format(mean_rmse_te_lr))
print("Mean R^2 (Train): {:.4f}".format(mean_r2_tr_lr))
print("Mean R^2 (Test): {:.4f}".format(mean_r2_te_lr))

Mean RMSE (Train): 0.5424
Mean RMSE (Test): 0.6114
Mean R^2 (Train): 0.7693
Mean R^2 (Test): 0.6805


In [10]:
mean_r2_tr_lr = np.mean(r2_tr_lr_list)
mean_r2_te_lr = np.mean(r2_te_lr_list)
# Define a function for MAPE (assuming you have this function defined)
# Set up the plot with a light blue background
plt.rcParams['figure.figsize'] = (2.0, 2.0)
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.dpi'] = 600

ax = plt.axes()
ax.set_facecolor("white")  # Light blue color
plt.grid()
plt.tick_params(grid_color='y', grid_alpha=0.6, grid_linewidth=0.3, grid_linestyle=':')
ax.spines['bottom'].set_linewidth('0.5')
ax.spines['bottom'].set_linestyle("-")
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth('0.5')
ax.spines['left'].set_linestyle("-")
ax.spines['left'].set_color('black')
ax.spines['top'].set_linewidth('0.5')
ax.spines['top'].set_linestyle("-")
ax.spines['top'].set_color('black')
ax.spines['right'].set_linewidth('0.5')
ax.spines['right'].set_linestyle("-")
ax.spines['right'].set_color('black')

# Set the x-axis and y-axis to a logarithmic scale with intervals of 10
new_ticks = list(np.arange(0, 15, 1))
plt.xticks(new_ticks, fontsize=4,fontweight='bold')
plt.yticks(new_ticks, fontsize=4,fontweight='bold')
plt.plot([0, 9.1], [0, 9.1], 'k--', linewidth='0.2', alpha=0.9)
# Plot the training set with circles and testing set with triangles
plt.scatter(train_labels, train_predicted_creeplife, alpha=0.9, s=2, linewidths=0.2, c='#00ffFF',
            edgecolor='#000000', label='training set' + ' ' + '$R^2$=%.4f' % mean_r2_tr_lr, marker='o')
plt.scatter(test_labels, test_predicted_creeplife, alpha=0.9, s=3, linewidths=0.2, c='#FF0000',
            edgecolor='#000000', label='testing set' + ' ' + '$R^2$=%.4f' % mean_r2_te_lr, marker='s')

plt.ylim(0,9.1)
plt.xlim(0,9.1)
# 添加图例
plt.legend(prop={'family': 'Times New Roman', 'size': 5, 'weight': 'bold'}, ncol=1)
# 显示性能指标
# Display the plot
plt.xlabel('lg(Real Creep Life)', size=5,weight='bold')
plt.ylabel('lg(Predicted Creep Life)', size=5,weight='bold')
plt.tick_params(which='major', direction='in', length=1.5, width=0.5, color='black', top=True, right=True)
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')
plt.text(7.5, 2.0, 'L R', fontsize=5,weight='bold')
plt.text(7.6, 1.5, '(d)', fontsize=5,weight='bold')
plt.show()